In [19]:
# # Python Version:
# !python --version

# # Pip:
# !pip install --disable-pip-version-check -q pip --upgrade > /dev/null
# !pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

# # AWS CLI:
# !pip install --disable-pip-version-check -q awscli==1.18.216 boto3==1.16.56 botocore==1.19.56

# # AWS SageMaker:
# !pip install --disable-pip-version-check -q sagemaker==2.29.0
# !pip install --disable-pip-version-check -q smdebug==1.0.1
# !pip install --disable-pip-version-check -q sagemaker-experiments==0.1.26

# # AWS Redshift:
# !pip install --disable-pip-version-check -q SQLAlchemy==1.3.22
# !pip install --disable-pip-version-check -q psycopg2-binary==2.9.1

# # Zip:
# !conda install -y zip

# # Pip List:
# !pip list

In [21]:
import boto3
import os
import pandas as pd
import sagemaker
import time
from time import gmtime, strftime

In [22]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

iam = boto3.client("iam", config=config)

# Getting Our Role Name of LabRole:
role_name = role.split("/")[-1]

print("Role name: {}".format(role_name))

# Checking permissions of our role:
admin = False
post_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]
for post_policy in post_policies:
    if post_policy["PolicyName"] == "AdministratorAccess":
        admin = True
        setup_iam_roles_passed = True
        print("[OK]")
        break
    else:
        print("*************** [ERROR] SageMakerExecutionRole needs the AdministratorAccess policy attached. *****************")

Role name: LabRole
*************** [ERROR] SageMakerExecutionRole needs the AdministratorAccess policy attached. *****************
*************** [ERROR] SageMakerExecutionRole needs the AdministratorAccess policy attached. *****************
*************** [ERROR] SageMakerExecutionRole needs the AdministratorAccess policy attached. *****************
*************** [ERROR] SageMakerExecutionRole needs the AdministratorAccess policy attached. *****************


In [23]:
# Start with Creation of the S3 Bucket

s3 = boto3.Session().client(service_name="s3", region_name=region)

setup_s3_bucket_passed = False

print("Default bucket: {}".format(bucket))

Default bucket: sagemaker-us-east-1-651000550833


In [24]:
%%bash

aws s3 ls s3://${bucket}/

2022-04-02 03:22:03 sagemaker-studio-651000550833-liw8vief93
2022-04-02 03:41:00 sagemaker-us-east-1-651000550833


In [25]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': 'G5V334D8MR7T72AR', 'HostId': 'Q7/zlOgPiRSIyVQenoEASAspWbcV0Lzt14o4Ab+KqXzP2CqBmzK233KSrygUUQfLyRhofAYQLJM=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'Q7/zlOgPiRSIyVQenoEASAspWbcV0Lzt14o4Ab+KqXzP2CqBmzK233KSrygUUQfLyRhofAYQLJM=', 'x-amz-request-id': 'G5V334D8MR7T72AR', 'date': 'Sun, 03 Apr 2022 18:01:44 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


In [26]:
break

SyntaxError: 'break' outside loop (<ipython-input-26-6aaf1f276005>, line 4)

Please upload FOUNDATION FOLDER into Default Bucket.

In [27]:
s3_bucket = pd.DataFrame.from_dict(s3.list_objects(Bucket=bucket)['Contents'])

# delete top 2 rows & reset index
s3_bucket = s3_bucket.iloc[2:,:].reset_index(drop=True)
s3_bucket.head(3)

,Key,LastModified,ETag,Size,StorageClass,Owner
0,Foundation/DataPrep/Instructions.txt,2022-04-03 17:51:18+00:00,"""263c481225c43eb861352ae0f153393c""",251,STANDARD,{'DisplayName': 'awslabsc0w1962079t1615546215'...
1,Foundation/DataPrep/Population_DataPrep.ipynb,2022-04-03 17:51:19+00:00,"""8f4bfe273c9f1d8ca1c2d2592607b35c""",25468,STANDARD,{'DisplayName': 'awslabsc0w1962079t1615546215'...
2,Foundation/DataPrep/SDGE_DataPrep.ipynb,2022-04-03 17:51:19+00:00,"""f9ce881a20a9dbfec3fad3d1d1f9bd75""",21427,STANDARD,{'DisplayName': 'awslabsc0w1962079t1615546215'...


In [29]:
# Base directories
root = '/root/'
raw = 'Raw Data'
clean = 'Clean_Data'

extensions = ['.csv', '.pdf', '.xlsx', '.py', '.ipynb']

create class dir
clean_dir = root + clean
os.mkdir(clean_dir)
os.chdir(clean_dir)

for index, row in s3_bucket.iterrows():
    file_dir = row['Key']
    
    # print every 10th index
    if index % 50 == 0:
        print(f'Processing {index}/{s3_bucket.shape[0]}')
        
    # dir_comprehension
    dir_comp = []

    dir_comp.append('/root')
    
    folder_dir = file_dir.split('/')

    for dir_ in folder_dir:
        # slowly rebuild folder directory 
        dir_comp.append(dir_)

        # The following checks if path exists
        current_dir_path = os.path.join(*dir_comp)
        
        
        # only create paths for non extension files
        if any(ext in current_dir_path for ext in extensions) != True:
            if os.path.isdir(current_dir_path) == False:
                os.mkdir(current_dir_path)
                os.chdir(current_dir_path)
        else:            
            # remove file_name.ext from current_dir_path by the following:
            # break down path into list 
            dir_path = current_dir_path.split('/')
            
            # get last element file_name.ext
            file_name = dir_path.pop()
            
            # rebuild list back to path
            dir_path = '/' + os.path.join(*dir_path)
            
            if (dir_path== os.getcwd()) != True:
                # aws needs root to be encased in '/'
                os.chdir(dir_path)
                
            # checks if file is one of the accepted extensions
            if any(ext in current_dir_path for ext in extensions) == True:
                s3.download_file(Bucket=bucket, Filename=file_name, Key=file_dir)  
                
    # complete
print('Ingestion is complete!')

Processing 0/946
Processing 50/946
Processing 100/946
Processing 150/946
Processing 200/946
Processing 250/946
Processing 300/946
Processing 350/946
Processing 400/946
Processing 450/946
Processing 500/946
Processing 550/946
Processing 600/946
Processing 650/946
Processing 700/946
Processing 750/946
Processing 800/946
Processing 850/946
Processing 900/946
Ingestion is complete!


# Data Processing

In [38]:
clean_dir = root + clean
os.mkdir(clean_dir)
os.chdir(clean_dir)

In [3]:
%run /root/Foundation/DataPrep/Population_DataPrep.ipynb
%run /root/Foundation/DataPrep/SDGE_DataPrep.ipynb
%run /root/Foundation/DataPrep/Weather_DataPrep.ipynb
%run /root/Foundation/DataPrep/Vehicle_DataPrep.ipynb

print('!!'*30)
print('All Data is ready to rock!')

Zip     object
Est     object
MOE     object
Year    object
dtype: object 

Zip     object
Est     object
MOE     object
Year    object
dtype: object 

Population Data is READY TO ROCK!!!!
Shape (29250, 8)
 ----------------------------------------
Shape (68760, 8)
 ----------------------------------------
SDGE Data is READY TO ROCK!!!!
Weather Data is READY TO ROCK!!!!
Shape (175291, 4)
 ----------------------------------------
Vehicle Data is READY TO ROCK!!!!
------------------------------------------------------------
All Data is ready to rock!
